In [ ]:
%%bash
rm -rf *

gdown "1fqx3CFdsuOxwW2hjMRhviSUOIzwBlV-J"
unzip dev_phase.zip > /dev/null

git clone "https://github.com/Toavina00/Polar-SemEval-2026-Task-9.git"
mv Polar-SemEval-2026-Task-9/src .

In [ ]:
import os
import torch
import numpy as np
import pandas as pd

import torch._dynamo
torch._dynamo.config.suppress_errors = True

import time
import tracemalloc
import pprint

from sklearn.metrics import f1_score

from transformers import (
    AutoTokenizer,
    AutoModel,
    DataCollatorWithPadding,
    EarlyStoppingCallback,
)

from src.dataset import PolarDataset, load_dataset
from src.model import PolarModel, PolarTrainer, PolarTrainingArgs
from src.utils import compile_submission

In [ ]:
LANGUAGES = ["eng", "hau", "swa"]
ROOT_DIR = "./"

def init_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)

device = "cuda" if torch.cuda.is_available() else "cpu"

checkpoint = "jhu-clsp/mmBERT-base"

In [ ]:
def compute_metrics(p):
    probs = torch.sigmoid(torch.from_numpy(p.predictions))
    preds = (probs > 0.5).int().numpy()
    return {'f1_macro': f1_score(p.label_ids, preds, average='macro')}

SUBTASK_ID = 1
BATCH_SIZE = 64

train_dir = os.path.join(ROOT_DIR, f"subtask{SUBTASK_ID}", "train")
dev_dir = os.path.join(ROOT_DIR, f"subtask{SUBTASK_ID}", "dev")

experiment_results = {}

dataset = load_dataset(
    train_dir,
    LANGUAGES,
    tokenizer,
    train=True,
    val_size=0.2,
    random_state=42,
    stratify=True,
)

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
training_args = PolarTrainingArgs(
    output_dir=f"./",
    num_train_epochs=10,
    learning_rate=2e-5,
    weight_decay=0.01,
    lr_scheduler_type='linear',
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    fp16=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    logging_steps=100,
    disable_tqdm=False,
    report_to="none",
)

for hidden_layers in [[], [256], [512], [1024], [256, 256], [512, 512], [1024, 1024]]:

    init_seed(42)

    experiment_label = f"{checkpoint.split('/')[-1]}-{hidden_layers}"
    
    model = PolarModel(checkpoint, len(dataset["labels"]), hidden_layers)
    model.to(device)
    
    trainer = PolarTrainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train_dataset"],
        eval_dataset=dataset["val_dataset"],    
        compute_metrics=compute_metrics,
        data_collator=DataCollatorWithPadding(tokenizer)
    )
    
    trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=3))
    
    trainer.train()
    
    eval_results = trainer.evaluate()
    
    experiment_results[experiment_label] = eval_results
    
    compile_submission(
        f'{experiment_label}.zip',
        data_dir=dev_dir,
        output_dir=os.path.join(ROOT_DIR, f"subtask_{SUBTASK_ID}"),
        languages=LANGUAGES,
        model=model,
        thresh=0.5,
        tokenizer=tokenizer,
        batch_size=BATCH_SIZE,
        device=device,
    )

In [ ]:
training_args = PolarTrainingArgs(
    output_dir=f"./",
    num_train_epochs=10,
    learning_rate=2e-5,
    layer_wise = True,
    layer_wise_decay = True,
    classifier_lr = 1e-4,
    weight_decay=0.01,
    lr_scheduler_type='linear',
    fp16=True,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    logging_steps=100,
    disable_tqdm=False,
    report_to="none",
)

init_seed(42)

experiment_label = f"{checkpoint.split('/')[-1]}-layer_wise"


model = PolarModel(checkpoint, len(dataset["labels"]), hidden_layers)
model.to(device)

trainer = PolarTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train_dataset"],
    eval_dataset=dataset["val_dataset"],    
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer)
)

trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=3))

trainer.train()

eval_results = trainer.evaluate()

experiment_results[experiment_label] = eval_results

compile_submission(
    f'{experiment_label}.zip',
    data_dir=dev_dir,
    output_dir=os.path.join(ROOT_DIR, f"subtask_{SUBTASK_ID}"),
    languages=LANGUAGES,
    model=model,
    thresh=0.5,
    tokenizer=tokenizer,
    batch_size=BATCH_SIZE,
    device=device,
)

In [ ]:
pd.DataFrame(experiment_results).to_json("results.json")

In [ ]:
%%bash

rm -rf dev_phase.zip
rm -rf Polar-SemEval-2026-Task-9 src
rm -rf subtask* checkpoint*
rm -rf run*